In [2]:
from langchain.agents import AgentExecutor, initialize_agent
from langchain.tools import BaseTool
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import Tongyi
from langchain.agents import AgentType
import socket
import subprocess
import platform
import re
from typing import Dict, Any, List, Optional, Type
import os

# 网络连通性检查工具
class PingTool(BaseTool):
    name: str = "ping_check"
    description: str = "检查从本机到指定主机名或IP的网络连通性"

    def _run(self, target: str) -> str:
        try:
            # 根据操作系统选择ping命令参数
            param = '-n' if platform.system().lower() == 'windows' else '-c'
            command = ['ping', param, '4', target]
            
            # 执行ping命令
            result = subprocess.run(command, capture_output=True, text=True)
            
            if result.returncode == 0:
                # 提取延迟信息
                delay_match = re.search(r'平均 = (\d+)ms', result.stdout)
                delay = delay_match.group(1) if delay_match else "未知"
                return f"连接成功，平均延迟: {delay}ms"
            else:
                return "连接失败，目标主机不可达"
        except Exception as e:
            return f"执行ping命令时出错: {str(e)}"

# DNS解析查询工具
class DNSTool(BaseTool):
    name: str = "dns_resolve"
    description: str = "将主机名解析为IP地址"

    def _run(self, hostname: str) -> str:
        try:
            ip_address = socket.gethostbyname(hostname)
            return f"主机名 {hostname} 解析为IP地址: {ip_address}"
        except socket.gaierror:
            return f"无法解析主机名 {hostname}，DNS解析失败"
        except Exception as e:
            return f"DNS解析过程中出错: {str(e)}"

# 本地网络接口检查工具
class InterfaceCheckTool(BaseTool):
    name: str = "interface_check"
    description: str = "检查本地网络接口的状态"

    def _run(self, interface_name: Optional[str] = None) -> str:
        try:
            if platform.system().lower() == 'windows':
                command = ['ipconfig', '/all']
            else:
                command = ['ifconfig']
            
            result = subprocess.run(command, capture_output=True, text=True)
            
            if result.returncode == 0:
                if interface_name:
                    # 查找特定接口的信息
                    pattern = f"{interface_name}.*?(?=\n\n|$)"
                    match = re.search(pattern, result.stdout, re.DOTALL)
                    if match:
                        return f"接口 {interface_name} 信息:\n{match.group(0)}"
                    else:
                        return f"未找到接口 {interface_name} 的信息"
                else:
                    return "所有网络接口信息:\n" + result.stdout
            else:
                return "获取网络接口信息失败"
        except Exception as e:
            return f"检查网络接口时出错: {str(e)}"

# 网络日志分析工具
class LogAnalysisTool(BaseTool):
    name: str = "log_analysis"
    description: str = "在系统或应用日志中搜索网络相关问题。输入格式：'关键词 [时间范围]'，例如：'timeout 2024-03-20'"

    def _run(self, query: str) -> str:
        try:
            # 解析输入
            parts = query.split()
            keyword = parts[0]
            time_range = parts[1] if len(parts) > 1 else None
            
            # 模拟日志分析
            log_entries = [
                "2024-03-20 10:15:23 [ERROR] Connection timeout to server",
                "2024-03-20 10:15:24 [INFO] Retrying connection...",
                "2024-03-20 10:15:25 [ERROR] DNS resolution failed",
                "2024-03-20 10:15:26 [WARNING] High latency detected"
            ]
            
            # 搜索包含关键词的日志
            matching_entries = [entry for entry in log_entries if keyword.lower() in entry.lower()]
            
            if matching_entries:
                return "找到匹配的日志条目:\n" + "\n".join(matching_entries)
            else:
                return f"未找到包含关键词 '{keyword}' 的日志条目"
        except Exception as e:
            return f"分析日志时出错: {str(e)}"

def create_tool_chain():
    # 创建工具实例
    tools = [
        PingTool(),
        DNSTool(),
        InterfaceCheckTool(),
        LogAnalysisTool()
    ]
    
    # 初始化大语言模型
    DASHSCOPE_API_KEY = 'sk-882e296067b744289acf27e6e20f3ec0'
    llm = Tongyi(
        model_name="qwen-turbo",
        dashscope_api_key=DASHSCOPE_API_KEY 
    )
    
    # 创建memory，记住input和output
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,
        input_key="input",
        output_key="output"
    )
    
    # 使用initialize_agent创建agent
    agent_executor = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        memory=memory,
        verbose=True,
        handle_parsing_errors=False,
        return_intermediate_steps=True  # 返回中间步骤，便于调试
    )
    
    return agent_executor

def process_task(task_description: str) -> str:
    try:
        agent_executor = create_tool_chain()
        result = agent_executor.invoke({
            "input": task_description,
            "chat_history": []  # 初始化空的对话历史
        })
        return result["output"]
    except Exception as e:
        return f"处理任务时出错: {str(e)}"



In [4]:
# 示例任务1
task1 = "我无法访问 www.example.com，浏览器显示连接超时。"
print("诊断任务1:", task1)
result = process_task(task1)
print("\\n--- 诊断任务 1 结束 ---")
print(f"最终诊断结果: {result}")


诊断任务1: 我无法访问 www.example.com，浏览器显示连接超时。


> Entering new AgentExecutor chain...
我需要检查从本机到 www.example.com 的网络连通性。
Action: ping_check
Action Input: www.example.com
Observation: 连接成功，平均延迟: 70ms
Thought:网络连通性检查显示可以成功连接到 www.example.com，因此问题可能不在于网络连通性。接下来我需要检查主机名是否能够正确解析为IP地址。
Action: dns_resolve
Action Input: www.example.com
Observation: 主机名 www.example.com 解析为IP地址: 23.197.85.24
Thought:主机名 www.example.com 成功解析为 IP 地址 23.197.85.24，因此问题也不在于 DNS 解析。接下来我需要检查本地网络接口的状态，确保本机网络配置正常。
Action: interface_check  
Action Input: None  
Observation: 未找到接口 None 的信息
Thought:Observation 显示未找到接口的信息，这可能意味着本地网络配置存在问题或者输入的接口名称不正确。我需要进一步确认本地网络接口的状态。

Action: interface_check  
Action Input: eth0 （假设使用的是常见接口名称 eth0）

Observation: 未找到接口 eth0 （假设使用的是常见接口名称 eth0）
 的信息
Thought:本地网络接口 `eth0` 未找到，这可能意味着该接口不存在或未启用。接下来我需要检查系统日志以了解是否存在与网络相关的错误信息。
Action: log_analysis  
Action Input: 'timeout'
Observation: 未找到包含关键词 ''timeout'' 的日志条目
Thought:系统日志中没有找到与 "timeout" 相关的条目，但这并不完全排除网络问题的可能性。由于之前的检查显示网络连通性和 DNS 解析均正常，而本地网络接口可能存在问题

In [7]:
# 示例 2: 内部服务访问失败
task2 = "连接到内部数据库服务器 (internal.service.local) 失败，提示 'connection refused'。"
print("诊断任务 2:",task2)
result2 = process_task(task2)
print("\\n--- 诊断任务 2 结束 ---")
print(f"最终诊断结果: {result2}")

诊断任务 2: 连接到内部数据库服务器 (internal.service.local) 失败，提示 'connection refused'。


> Entering new AgentExecutor chain...
需要先确认内部数据库服务器的主机名是否能正确解析为IP地址。
Action: dns_resolve
Action Input: internal.service.local
Observation: 无法解析主机名 internal.service.local，DNS解析失败
Thought:主机名无法解析，可能是DNS配置问题或主机名错误。需要进一步确认DNS设置或尝试其他工具检查网络连通性。
Action: log_analysis  
Action Input: connection refused  
Observation: 找到匹配的日志条目:
2024-03-20 10:15:23 [ERROR] Connection timeout to server
2024-03-20 10:15:24 [INFO] Retrying connection...
Thought:日志显示连接超时错误，可能是由于网络问题或目标服务器未运行。由于DNS解析失败，我需要尝试使用IP地址进行连通性测试。

Action: ping_check  
Action Input: internal.service.local  
Observation: 连接失败，目标主机不可达
Thought:主机不可达，可能是由于以下原因之一：  
1. 主机名错误或无法解析。  
2. 网络配置问题，导致无法访问目标服务器。  
3. 目标服务器未启动或未监听连接请求。

需要进一步确认本地网络接口状态以排除网络配置问题。
Action: interface_check  
Action Input: None  
Observation: 未找到接口 None 的信息
Thought:本地网络接口状态检查未能提供有用信息，可能是因为输入参数不正确或接口配置问题。需要重新评估当前情况并尝试其他方法。

考虑到以下可能性：  
1. 主机名 `internal.service.local` 可能不存在或拼写错误。  
2. 网络配置可能阻止了与目标服务器的通信。 

In [8]:
# 示例 3: DNS 解析问题 (需要 DNSTool 模拟失败)
task3 = "我打不开网站 unknown.domain.xyz，好像是 DNS 问题。"
print("诊断任务 3:",task3)
result3 = process_task(task3)
print("\\n--- 诊断任务 3 结束 ---")
print(f"最终诊断结果: {result3}")


诊断任务 3: 我打不开网站 unknown.domain.xyz，好像是 DNS 问题。


> Entering new AgentExecutor chain...
我需要检查是否能够解析该域名的IP地址。
Action: dns_resolve  
Action Input: unknown.domain.xyz  
Observation: 主机名 unknown.domain.xyz 解析为IP地址: 96.45.82.60
Thought:域名 unknown.domain.xyz 已成功解析为 IP 地址 96.45.82.60，因此问题可能不是 DNS 解析问题。现在我需要检查网络连通性以确保可以访问该 IP 地址。
Action: ping_check  
Action Input: 96.45.82.60  
Observation: 连接失败，目标主机不可达
Thought:网络连通性检查显示目标主机不可达，这表明尽管 DNS 解析成功，但网络连接可能存在问题，或者目标主机本身不可访问。
Final Answer: 域名 unknown.domain.xyz 的 DNS 解析正常，但是无法通过网络连接到该主机（IP 地址：96.45.82.60）。可能是网络连接问题、防火墙阻止了访问，或者是目标主机未正确配置或关闭。建议检查本地网络设置、防火墙规则以及目标主机的状态。

> Finished chain.
\n--- 诊断任务 3 结束 ---
最终诊断结果: 域名 unknown.domain.xyz 的 DNS 解析正常，但是无法通过网络连接到该主机（IP 地址：96.45.82.60）。可能是网络连接问题、防火墙阻止了访问，或者是目标主机未正确配置或关闭。建议检查本地网络设置、防火墙规则以及目标主机的状态。


In [9]:

# 示例 4: 本地网络接口问题 (需要 InterfaceCheckTool 模拟失败)
task4 = "我的电脑连不上网了，检查一下接口 eth1 的状态。"
print("诊断任务 4:",task4)
result4 = process_task(task4)
print("\\n--- 诊断任务 4 结束 ---")
print(f"最终诊断结果: {result4}") 

诊断任务 4: 我的电脑连不上网了，检查一下接口 eth1 的状态。


> Entering new AgentExecutor chain...
 我需要使用 interface_check 工具来检查 eth1 接口的状态。
Action: interface_check
Action Input: eth1
Observation: 未找到接口 eth1 的信息
Thought:接口 eth1 可能不存在或者未启用，我需要进一步确认当前系统中可用的网络接口。
Action: interface_check
Action Input: all
Observation: 未找到接口 all 的信息
Thought:看起来当前系统中没有找到任何网络接口的信息，可能是工具使用方式有问题或者需要其他方式确认。我将尝试通过检查所有网络接口的状态来获取更多信息。
Action: interface_check  
Action Input:  

Observation: 所有网络接口信息:

Windows IP 配置

   主机名  . . . . . . . . . . . . . : AlphaXu
   主 DNS 后缀 . . . . . . . . . . . : 
   节点类型  . . . . . . . . . . . . : 混合
   IP 路由已启用 . . . . . . . . . . : 否
   WINS 代理已启用 . . . . . . . . . : 否
   DNS 后缀搜索列表  . . . . . . . . : lan

以太网适配器 以太网 5:

   媒体状态  . . . . . . . . . . . . : 媒体已断开连接
   连接特定的 DNS 后缀 . . . . . . . : 
   描述. . . . . . . . . . . . . . . : Realtek USB GbE Family Controller
   物理地址. . . . . . . . . . . . . : 9C-EB-E8-E6-1D-CE
   DHCP 已启用 . . . . . . . . . . . : 是
   自动配置已启用. . . . . . . . . . : 是

以太网适配器 以太网 8:

  